# Import Libraries

In [1]:
# Import libraries
import mne
import json
import numpy as np
import pandas as pd
import scipy.signal as signal
import matplotlib.pyplot as plt

# Import custom scripts
from Functions import import_data
from Functions import data_tools
from Functions import processing

# Enable interactive plots
%matplotlib qt

# Import Data

Import EEG data
- Set channel names
- Create an mne raw object
- Set MNE channel montage

In [2]:
# import EEG data and fill settings
participant_id = "P003"
file_name = f"Pilot2\EEG\sub-{participant_id}\ses-S001\eeg\sub-{participant_id}_ses-S001_task-T1_run-001_eeg.xdf"
ch_names = ["Fz", "F4", "F8", "C3", "Cz", "C4", "T8", "P7", "P3", "P4", "P8", "PO7", "PO8", "O1", "Oz", "O2"]

[eeg_ts, eeg_data, eeg_fs] = import_data.read_xdf(f"Data\\{file_name}", picks=ch_names)

# Create MNE array
info = mne.create_info(ch_names, eeg_fs, ch_types = 'eeg')  # Create info properties
mne_raw = mne.io.RawArray(eeg_data, info = info)            

# Set standard channel montageR
mne_raw.set_montage('standard_1020')

<>:3: SyntaxWarning: invalid escape sequence '\E'
<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\E'
<>:3: SyntaxWarning: invalid escape sequence '\s'
C:\Users\admin\AppData\Local\Temp\ipykernel_25552\1703708234.py:3: SyntaxWarning: invalid escape sequence '\E'
  file_name = f"Pilot2\EEG\sub-{participant_id}\ses-S001\eeg\sub-{participant_id}_ses-S001_task-T1_run-001_eeg.xdf"
C:\Users\admin\AppData\Local\Temp\ipykernel_25552\1703708234.py:3: SyntaxWarning: invalid escape sequence '\s'
  file_name = f"Pilot2\EEG\sub-{participant_id}\ses-S001\eeg\sub-{participant_id}_ses-S001_task-T1_run-001_eeg.xdf"


Creating RawArray with float64 data, n_channels=16, n_times=199168
    Range : 0 ... 199167 =      0.000 ...   777.996 secs
Ready.


<RawArray | 16 x 199168 (778.0 s), ~24.3 MiB, data loaded>

In [ ]:
# Import comfort data
comfort_data = pd.read_csv(f"Data\\Pilot2\\Bracket\\{participant_id}\\{participant_id}_single_absolute.csv") 
comfort_data['Epoch'] = comfort_data['Epoch'] - 1 # Adjust epoch index to start from 0
grouped = comfort_data.groupby(["Contrast", "Size"])
    
# Create a dictionary for this subject: keys like "ContrastXSizeY" map to lists of comfort values
stim_comfort_dict = {}
for (contrast, size), group in grouped:
    key = f"Contrast{contrast}Size{size}"
    values = group["Comfort_Score"].tolist()
    stim_comfort_dict[key] = values
        
# Save the per-subject comfort dictionary for later processing
comfort_data_dict = stim_comfort_dict

# Divide into different signal arrays to apply filters

In [4]:
delta_raw = mne_raw.copy().filter(l_freq=1, h_freq=4, picks=ch_names)
theta_raw = mne_raw.copy().filter(l_freq=4, h_freq=8, picks=ch_names)
alpha_raw = mne_raw.copy().filter(l_freq=8, h_freq=12, picks=ch_names)
beta_raw = mne_raw.copy().filter(l_freq=12, h_freq=30, picks=ch_names)

alpha_raw = alpha_raw.copy().notch_filter(freqs = 10, picks=ch_names) 
beta_raw = beta_raw.copy().notch_filter(freqs = 20, picks=ch_names)   

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 845 samples (3.301 s)



Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 423 samples (1.652 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00

# Epoch data for each band

In [ ]:
# Settings
list_of_events = []

for x in range(4):
    for y in range(3):
        list_of_events.append(f"Contrast{x+1}Size{y+1}")
epoch_end = "stimulus ended" 

# Create a dict of stimuli using the unique events
dict_of_stimuli = {i: event for i, event in enumerate(list_of_events)}

# Import markers
[marker_ts, markers] = import_data.read_xdf_unity_markers(f"Data\\{file_name}")   

# Create Individual epochs for each event
[delta_events_epochs, delta_eeg_epochs] = data_tools.create_epochs(eeg_data = delta_raw.get_data(), eeg_ts = eeg_ts, markers = markers, markers_ts = marker_ts, events = list_of_events, epoch_end = epoch_end)
[theta_events_epochs, theta_eeg_epochs] = data_tools.create_epochs(eeg_data = theta_raw.get_data(), eeg_ts = eeg_ts, markers = markers, markers_ts = marker_ts, events = list_of_events, epoch_end = epoch_end)
[alpha_events_epochs, alpha_eeg_epochs] = data_tools.create_epochs(eeg_data = alpha_raw.get_data(), eeg_ts = eeg_ts, markers = markers, markers_ts = marker_ts, events = list_of_events, epoch_end = epoch_end)
[beta_events_epochs, beta_eeg_epochs]   = data_tools.create_epochs(eeg_data = beta_raw.get_data(),  eeg_ts = eeg_ts, markers = markers, markers_ts = marker_ts, events = list_of_events, epoch_end = epoch_end)

# Organize epochs by stimuli and frequency
delta_epochs_organized = data_tools.epochs_stim(eeg_epochs = delta_eeg_epochs, labels = delta_events_epochs, stimuli = dict_of_stimuli)
theta_epochs_organized = data_tools.epochs_stim(eeg_epochs = theta_eeg_epochs, labels = theta_events_epochs, stimuli = dict_of_stimuli)
alpha_epochs_organized = data_tools.epochs_stim(eeg_epochs = alpha_eeg_epochs, labels = alpha_events_epochs, stimuli = dict_of_stimuli)
beta_epochs_organized  = data_tools.epochs_stim(eeg_epochs = beta_eeg_epochs,  labels = beta_events_epochs,  stimuli = dict_of_stimuli)

# Make organized epochs into dictionaries

In [ ]:
delta_dict = {stim_label: np.array(delta_epochs_organized[stim_idx]) for stim_idx, stim_label in dict_of_stimuli.items()}
theta_dict = {stim_label: np.array(theta_epochs_organized[stim_idx]) for stim_idx, stim_label in dict_of_stimuli.items()}
alpha_dict = {stim_label: np.array(alpha_epochs_organized[stim_idx]) for stim_idx, stim_label in dict_of_stimuli.items()}
beta_dict  = {stim_label: np.array(beta_epochs_organized[stim_idx])  for stim_idx, stim_label in dict_of_stimuli.items()}

# Compute PSD for each band

In [ ]:
window_size = 5

delta_eeg_f, delta_eeg_pxx = processing.psd(delta_dict, eeg_fs, window_size)
theta_eeg_f, theta_eeg_pxx = processing.psd(theta_dict, eeg_fs, window_size)
alpha_eeg_f, alpha_eeg_pxx = processing.psd(alpha_dict, eeg_fs, window_size)
beta_eeg_f,  beta_eeg_pxx  = processing.psd(beta_dict,  eeg_fs, window_size)

# Get the mean PSD for each epoch (for each band)

In [ ]:
delta_mean_power_per_epoch = {}
theta_mean_power_per_epoch = {}
alpha_mean_power_per_epoch = {}
beta_mean_power_per_epoch  = {}

for stim_label, pxx_list in delta_eeg_pxx.items():
    delta_mean_power_per_epoch[stim_label] = [np.mean(pxx, axis = 1) for pxx in pxx_list]

for stim_label, pxx_list in theta_eeg_pxx.items():
    theta_mean_power_per_epoch[stim_label] = [np.mean(pxx, axis = 1) for pxx in pxx_list]

for stim_label, pxx_list in alpha_eeg_pxx.items():
    alpha_mean_power_per_epoch[stim_label] = [np.mean(pxx, axis = 1) for pxx in pxx_list]

for stim_label, pxx_list in beta_eeg_pxx.items():
    beta_mean_power_per_epoch[stim_label]  = [np.mean(pxx, axis = 1) for pxx in pxx_list]

# Calculate ratios

In [ ]:
#alpha/beta ratio
alpha_beta_ratio = {}

#theta/alpha ratio
theta_alpha_ratio = {}

#theta/beta ratio
theta_beta_ratio = {}

#theta + alpha
theta_alpha_sum = {}

#(theta + alpha) / beta
theta_alpha_by_beta_ratio = {}

#theta + alpha + beta
theta_alpha_beta_sum = {}   

#(alpha + theta)/(alpha + beta)
alpha_theta_by_alpha_beta_ratio = {}

#theta/(alpha + beta
theta_by_alpha_beta_ratio = {}

for stim_label in dict_of_stimuli.values()

# Combine & Export

In [ ]:
records = []

all_band_data = {
    'Delta': delta_mean_power_per_epoch,
    'Theta': theta_mean_power_per_epoch,
    'Alpha': alpha_mean_power_per_epoch,
    'Beta' : beta_mean_power_per_epoch
}

# Iterate through epochs and stimuli (assuming all bands have same structure)
for stim_label in list(all_band_data.values())[0].keys():
    for epoch_idx in range(len(list(all_band_data.values())[0][stim_label])):
        for comfort_idx, (c_stim_label, comfort_score) in enumerate(comfort_data_dict.items()):
            if stim_label == c_stim_label:
                comfort_value = comfort_score[epoch_idx]

                row = {
                    'Participant': participant_id,
                    'Stimulus': stim_label,
                    'Epoch': epoch_idx,
                    'Comfort Score': comfort_value 
                }

                for band_name, band_dict in all_band_data.items():
                    pxx = band_dict[stim_label][epoch_idx]  # shape: (channels,)
                    for ch_idx, ch_name in enumerate(ch_names):
                        col_name = f"{ch_name}_{band_name.lower()}"
                        row[col_name] = pxx[ch_idx]
                records.append(row)

df_wide = pd.DataFrame(records)
df_wide.to_csv("eeg_band_power_wide_format.csv", index=False)